In [4]:
import nltk
from nltk.corpus import reuters

import numpy as np
import random
import re
import time

import torch
import torch.nn as nn
import torch.optim as optim

from scipy.stats import spearmanr
from collections import Counter

In [5]:
def build_corpus():
    corpus = []
    for id in reuters.fileids():
        words = reuters.words(id)
        words = [word.lower() for word in words if word.isalpha()]
        corpus.append(words)

    return corpus


corpus = build_corpus()
corpus, len(corpus)  # 10788 sentences

([['asian',
   'exporters',
   'fear',
   'damage',
   'from',
   'u',
   's',
   'japan',
   'rift',
   'mounting',
   'trade',
   'friction',
   'between',
   'the',
   'u',
   's',
   'and',
   'japan',
   'has',
   'raised',
   'fears',
   'among',
   'many',
   'of',
   'asia',
   's',
   'exporting',
   'nations',
   'that',
   'the',
   'row',
   'could',
   'inflict',
   'far',
   'reaching',
   'economic',
   'damage',
   'businessmen',
   'and',
   'officials',
   'said',
   'they',
   'told',
   'reuter',
   'correspondents',
   'in',
   'asian',
   'capitals',
   'a',
   'u',
   's',
   'move',
   'against',
   'japan',
   'might',
   'boost',
   'protectionist',
   'sentiment',
   'in',
   'the',
   'u',
   's',
   'and',
   'lead',
   'to',
   'curbs',
   'on',
   'american',
   'imports',
   'of',
   'their',
   'products',
   'but',
   'some',
   'exporters',
   'said',
   'that',
   'while',
   'the',
   'conflict',
   'would',
   'hurt',
   'them',
   'in',
   'the',


In [10]:
def build_vocab(corpus):
    # Flatten words from all sentences
    flatten_words = [word for sentence in corpus for word in sentence]
    word_counts = Counter(flatten_words)

    # Unique words
    vocab = list(set(flatten_words))
    vocab.append("<UNKNOWN>")

    word2index = {word: index for index, word in enumerate(vocab)}
    # word2index['<UNKNOWN>'] = 0
    # word2index

    return vocab, len(vocab), word2index, word_counts


vocab, vocab_size, word2index, word_counts = build_vocab(corpus)
vocab, vocab_size  # 29174 unique words

(['revamped',
  'technitrol',
  'luso',
  'cuban',
  'erection',
  'uspts',
  'cip',
  'aleging',
  'foiled',
  'applications',
  'reassessment',
  'dane',
  'shirmax',
  'tierco',
  'hodeidah',
  'wjye',
  'miles',
  'liable',
  'wedd',
  'hachette',
  'tragedy',
  'meditrust',
  'revenueslast',
  'maioled',
  'computing',
  'donald',
  'blamed',
  'envisage',
  'ti',
  'warplnes',
  'travel',
  'financed',
  'vineyards',
  'declining',
  'forecaster',
  'devote',
  'belligerent',
  'supd',
  'violators',
  'pump',
  'utility',
  'pooling',
  'frequency',
  'salads',
  'specified',
  'zirconium',
  'unilever',
  'conc',
  'voytko',
  'metre',
  'ubks',
  'corportation',
  'observed',
  'drh',
  'gevirtz',
  'swingeing',
  'feeder',
  'nva',
  'mexican',
  'recovery',
  'kaltenbacher',
  'takeuchi',
  'natned',
  'geophysicists',
  'comprising',
  'conglommerate',
  'took',
  'hein',
  'datachecker',
  'avoid',
  'bayerische',
  'hedging',
  'leader',
  'hernandez',
  'telkon',
  'prog

In [7]:
def build_skipgrams(corpus, word2index, window_size=2):
    skip_grams = []

    for sentence in corpus:
        for position, center_word in enumerate(sentence):
            center_index = word2index[center_word]
            context_indices = list(
                [
                    i
                    for i in range(
                        max(position - window_size, 0),
                        min(position + window_size + 1, len(sentence)),
                    )
                    if i != position
                ]
            )
            for index in context_indices:
                context_word = sentence[index]
                context_index = word2index[context_word]
                skip_grams.append((center_index, context_index))

    return skip_grams


skip_grams = build_skipgrams(corpus, word2index, window_size=2)
skip_grams, len(skip_grams)  # 5243836 pairs

([(19633, 3698),
  (19633, 1543),
  (3698, 19633),
  (3698, 1543),
  (3698, 14465),
  (1543, 19633),
  (1543, 3698),
  (1543, 14465),
  (1543, 25473),
  (14465, 3698),
  (14465, 1543),
  (14465, 25473),
  (14465, 17538),
  (25473, 1543),
  (25473, 14465),
  (25473, 17538),
  (25473, 26941),
  (17538, 14465),
  (17538, 25473),
  (17538, 26941),
  (17538, 9042),
  (26941, 25473),
  (26941, 17538),
  (26941, 9042),
  (26941, 27587),
  (9042, 17538),
  (9042, 26941),
  (9042, 27587),
  (9042, 11455),
  (27587, 26941),
  (27587, 9042),
  (27587, 11455),
  (27587, 188),
  (11455, 9042),
  (11455, 27587),
  (11455, 188),
  (11455, 24265),
  (188, 27587),
  (188, 11455),
  (188, 24265),
  (188, 7934),
  (24265, 11455),
  (24265, 188),
  (24265, 7934),
  (24265, 12768),
  (7934, 188),
  (7934, 24265),
  (7934, 12768),
  (7934, 17538),
  (12768, 24265),
  (12768, 7934),
  (12768, 17538),
  (12768, 26941),
  (17538, 7934),
  (17538, 12768),
  (17538, 26941),
  (17538, 9581),
  (26941, 12768),
  (

In [12]:
def build_unigram_table(word_counts, power=0.75):
    total_count = sum([count for count in word_counts.values()])
    unigram_table = []
    for word, count in word_counts.items():
        score = (count / total_count) ** power
        unigram_table.extend([word] * int(score * 1e6))
    return unigram_table


unigram_table = build_unigram_table(word_counts)
unigram_table

['asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',
 'asian',


In [ ]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, embed_size, mode='softmax'):
        super(Skipgram, self).__init__()
        
        self.mode = mode
        self.embedding_v = nn.Embedding(vocab_size, embed_size)
        self.embedding_u = nn.Embedding(vocab_size, embed_size)
        
    def forward(self, center_words, predicted_words,)